In [42]:
import random as rd
import numpy as np


#this just generates a random sequence of bitstream that is whatever length you want

def rand_bitstream(bitlength):
    bitstream = ""
    for i in range(bitlength):
        temp = str(rd.randint(0,1))
        bitstream += temp
    return bitstream



#because I was lazy and didnt want to write this over and over
def con_bin_str(input1):
    step1 = format(input1,'b')
    step2 = str(step1.zfill(96))#hella paranoid
    step3 = step2[:96]#we had cases of output being 97 bits long... can't have that
    return step3

#I wanted to define this so that you wouldn't get a random starting IDP_K data stream Every time 
# you did a protocal run.
IDP_K_str = '111111111000100100001110010001011010011111100010010000000111100101111101110110101101000011000111101101001011100000111010110011100001101110111001111111110100010010010001100000011101000110001001000101101001110010000010111110000011001100001100111101101011000010111100110101100110101000110101110001011010111110101010111100101001111001100101000111010110111101111100100011100010101011000010111110011111110011100000111010011011011100111000110100100110100110011011001011111101011011010110'


In [3]:

def MMAP(oracle):
    MOD_SIZE = 2**96
    
    #this is our ID, never changing, always the same
    ID_MMAP_str ='101110011011001011100100001101010011110010001001000101001101001001110010110100100001011001110010'
    
    ID_MMAP = int(ID_MMAP_str,2)
    
    IDP_MMAP_str, K_1_MMAP_str, K_2_MMAP_str, K_3_MMAP_str, K_4_MMAP_str = [oracle[i:i+96] for i in range(0,len(oracle),96)]
    
    IDP_MMAP = int(IDP_MMAP_str,2)
    K_1_MMAP = int(K_1_MMAP_str,2)
    K_2_MMAP = int(K_2_MMAP_str,2)
    K_3_MMAP = int(K_3_MMAP_str,2)
    K_4_MMAP = int(K_4_MMAP_str,2)
    
    
    n1_MMAP = int(rand_bitstream(96),2)
    n2_MMAP = int(rand_bitstream(96),2)
    
    A_int_MMAP = (IDP_MMAP ^ K_1_MMAP ^ n1_MMAP)%MOD_SIZE
    B_int_MMAP = ((IDP_MMAP & K_2_MMAP) | n1_MMAP)%MOD_SIZE
    C_int_MMAP = (IDP_MMAP + K_3_MMAP + n2_MMAP)%MOD_SIZE
    D_int_MMAP = ((IDP_MMAP | K_4_MMAP) & n2_MMAP)%MOD_SIZE
    E_int_MMAP = ((IDP_MMAP + ID_MMAP) ^ n1_MMAP)%MOD_SIZE
    
    NextIDP_int_MMAP = ((IDP_MMAP + (n1_MMAP ^ n2_MMAP)) ^ ID_MMAP)%MOD_SIZE
    NextK_1_int_MMAP = (K_1_MMAP ^ n1_MMAP ^ (K_3_MMAP + ID_MMAP))%MOD_SIZE
    NextK_2_int_MMAP = (K_2_MMAP ^ n2_MMAP ^ (K_4_MMAP + ID_MMAP))%MOD_SIZE
    NextK_3_int_MMAP = ((K_3_MMAP ^ n1_MMAP) + (K_1_MMAP ^ ID_MMAP))%MOD_SIZE
    NextK_4_int_MMAP = ((K_4_MMAP ^ n1_MMAP) + (K_2_MMAP ^ ID_MMAP))%MOD_SIZE
    
    A_MMAP = con_bin_str(A_int_MMAP)
    B_MMAP = con_bin_str(B_int_MMAP)
    C_MMAP = con_bin_str(C_int_MMAP)
    D_MMAP = con_bin_str(D_int_MMAP)
    E_MMAP = con_bin_str(E_int_MMAP)
    
    NextIDP_MMAP = con_bin_str(NextIDP_int_MMAP)
    NextK_1_MMAP = con_bin_str(NextK_1_int_MMAP)
    NextK_2_MMAP = con_bin_str(NextK_2_int_MMAP)
    NextK_3_MMAP = con_bin_str(NextK_3_int_MMAP)
    NextK_4_MMAP = con_bin_str(NextK_4_int_MMAP)
    
    NextIDP_K_MMAP = NextIDP_MMAP + NextK_1_MMAP + NextK_2_MMAP + NextK_3_MMAP + NextK_4_MMAP
    
    return A_MMAP,B_MMAP,C_MMAP,D_MMAP,E_MMAP,NextIDP_K_MMAP
    
    

    
AMMAP,BMMAP,CMMAP,DMMAP,EMMAP,NextIDP_KMMAP = MMAP(IDP_K_str)








In [4]:
def Fp_func(input1):
    
    chunks = [input1[i:i+4] for i in range(0,len(input1),4)]
    
    final_output_str = ''
    
    for j in range(len(chunks)):
        
        k = 0
        
        while k < len(chunks[j]):
                   
            x1_str, x2_str, x3_str, x4_str = [chunks[j][l:l+1] for l in range(0,len(chunks[j]),1)]
            
            x1 = int(x1_str,2)
            x2 = int(x2_str,2)
            x3 = int(x3_str,2)
            x4 = int(x4_str,2)
            
            output = ((x1 ^ x2) ^ (x3 ^ x4))
            
            output_binary = format(output,'b')
            output_str = str(output_binary)
            
            k = k + 1
            
        final_output_str += output_str
            
    return final_output_str    
   
    
    

In [29]:
#Finds and outputs the ID of an MMAP RFID system, given only intercepted comms between reader and tag
SIZE = 96
def MMAP_Cracker(runtime):
    garbage = "" #Used to collect useless information, and ignore it
    
    #Finding the first IDP we will use, so that it works from any random valid IDP
    #We don't care about the previous A-E, since we are assuming that we don't know the IDP used for that interaction
    garbage,garbage,garbage,garbage,garbage,IDP_next = MMAP(IDP_K_str)
    
    #Initializing our ID output to all 9s, to represent that we don't know what it is
    ID = np.array([9]*SIZE)
    
    #Until we return a complete ID, keep making MMAP exchanges
    while(1):
        runtime+=1
        #Initializing our IDP
        IDP = IDP_next[0:SIZE]
        #Initializing A-E, and the next IDP
        A,B,C,D,E,IDP_next = MMAP(IDP_next)
        
        #Initializing n1 to all 9s
        n1 = np.array([9]*SIZE)
        
        #Finding all knowable values of n1
        i = 0
        for i in range(SIZE):
            if IDP[i] == '0':
                if B[i] == '0':
                    n1[i] = 0
                else:
                    n1[i] = 1
       # npPrint(n1)
        #Creating our negative IDP in two's complement
        negIDP = ""
        for i in range(SIZE):
            if IDP[i] == "0":
                negIDP += "1"
            else:
                negIDP += "0"
        negIDP = bin(int(negIDP,2)+1)[2:].zfill(SIZE)
#         print("IDP:    ",IDP)
#         print("negIDP: ",negIDP)
#         print("IDPCHK: ", bin(int(IDP,2)+int(negIDP,2)))
        
        #Initializing E XOR n1 to 9s
        E_xor_n1 = np.array([9]*SIZE)
        
        #Calculating knowable values of E XOR n1
    #    print(E)
        for i in range(SIZE):
            num = int(E[i],2)
            if(n1[i] != 9):
                E_xor_n1[i] = num^n1[i]
    #    npPrint(E_xor_n1)
        
        carry = 0
        for i in range(SIZE-1,-1,-1):
            if ID[i] == 9: break
            
            # negIDP 111
            # EXOR 001
            # ID 000
            
            # ID 000
            # negIDP 111
            # EXOR 111 WRONG
            val = ID[i] + int(negIDP[i],2) + carry
            if E_xor_n1[i] == 9:
                E_xor_n1[i] = (val)%2
            carry = E_xor_n1[i] + int(negIDP[i],2) + carry > 1
        #print("E_xor_n1 predic: ",npPrint(E_xor_n1))
        
        #Calculating knowable values of the ID
        carry = 0
        for i in range(SIZE-1,-1,-1):
            if E_xor_n1[i] == 9: break
            val = E_xor_n1[i] + int(negIDP[i],2) + carry
            if ID[i] == 9:
                ID[i] = (val)%2
            carry = val > 1
        print(ID)
        #If ID's most significant bit is found, return it
        if ID[0] != 9:
            print(runtime)
            return ID
MMAP_Cracker(0)

[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 0]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 1 1 0 0 1 0]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 1 0 0 1 1 1 0 0 1 0]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 1 1 0 0 1 1 1 0 0 1 0]
[9 9 9 9 9 9 9 9 9 9

array([1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 0])

In [4]:
class EMAPoracle:
    def __init__(self, ID):
        self.ID = ID
        self.IDP = IDP_K_str
    
    def EMAP(oracle):
    MOD_SIZE = 2**96
    ID_EMAP_str ='101110011011001011100100001101010011110010001001000101001101001001110010110100100001011001110010'
    
    ID_EMAP = int(ID_EMAP_str,2)
    
    IDP_EMAP_str, K_1_EMAP_str, K_2_EMAP_str, K_3_EMAP_str, K_4_EMAP_str = [oracle[i:i+96] for i in range(0,len(oracle),96)]
    
    IDP_EMAP = int(IDP_EMAP_str,2)
    K_1_EMAP = int(K_1_EMAP_str,2)
    K_2_EMAP = int(K_2_EMAP_str,2)
    K_3_EMAP = int(K_3_EMAP_str,2)
    K_4_EMAP = int(K_4_EMAP_str,2)
    
    
    n1_EMAP = int(rand_bitstream(96),2)
    n2_EMAP = int(rand_bitstream(96),2)
    
    KI = K_1_EMAP ^ K_2_EMAP ^ K_3_EMAP ^ K_4_EMAP
    
    A_int_EMAP = (IDP_EMAP ^ K_1_EMAP ^ n1_EMAP)%MOD_SIZE
    B_int_EMAP = ((IDP_EMAP | K_2_EMAP) ^ n1_EMAP)%MOD_SIZE
    C_int_EMAP = (IDP_EMAP ^ K_3_EMAP ^ n2_EMAP)%MOD_SIZE # Changed
    D_int_EMAP = ((IDP_EMAP & K_4_EMAP) ^ n2_EMAP)%MOD_SIZE
    E_int_EMAP = ((IDP_EMAP & n1_EMAP | n2_EMAP) ^ ID_EMAP ^ KI)%MOD_SIZE
    
    ID_EMAP_str_1 , ID_EMAP_str_2 = [ID_EMAP_str[i:i+48] for i in range(0,len(ID_EMAP_str),48)]
    
    K_1_Fp_str = Fp_func(K_1_EMAP_str)
    K_2_Fp_str = Fp_func(K_2_EMAP_str)
    K_3_Fp_str = Fp_func(K_3_EMAP_str)
    K_4_Fp_str = Fp_func(K_4_EMAP_str)
    
    Sub_NextK_1_str = (ID_EMAP_str_1 + K_4_Fp_str + K_3_Fp_str)
    Sub_NextK_2_str = (K_1_Fp_str + K_4_Fp_str + ID_EMAP_str_2)
    Sub_NextK_3_str = (ID_EMAP_str_1 + K_4_Fp_str + K_2_Fp_str)
    Sub_NextK_4_str = (K_3_Fp_str + K_1_Fp_str + ID_EMAP_str_2)
    
    Sub_NextK_1 = int(Sub_NextK_1_str,2)
    Sub_NextK_2 = int(Sub_NextK_2_str,2)
    Sub_NextK_3 = int(Sub_NextK_3_str,2)
    Sub_NextK_4 = int(Sub_NextK_4_str,2)
    
    NextIDP_int_EMAP = (IDP_EMAP ^ n2_EMAP ^ K_1_EMAP )%MOD_SIZE
    NextK_1_int_EMAP = (K_1_EMAP ^ n2_EMAP ^ Sub_NextK_1)%MOD_SIZE
    NextK_2_int_EMAP = (K_2_EMAP ^ n2_EMAP ^ Sub_NextK_2)%MOD_SIZE
    NextK_3_int_EMAP = (K_3_EMAP ^ n1_EMAP ^ Sub_NextK_3)%MOD_SIZE
    NextK_4_int_EMAP = (K_4_EMAP ^ n1_EMAP ^ Sub_NextK_4)%MOD_SIZE
    
    
    A_EMAP = con_bin_str(A_int_EMAP)
    B_EMAP = con_bin_str(B_int_EMAP)
    C_EMAP = con_bin_str(C_int_EMAP)
    D_EMAP = con_bin_str(D_int_EMAP)
    E_EMAP = con_bin_str(E_int_EMAP)
    
    NextIDP_EMAP = con_bin_str(NextIDP_int_EMAP)
    NextK_1_EMAP = con_bin_str(NextK_1_int_EMAP)
    NextK_2_EMAP = con_bin_str(NextK_2_int_EMAP)
    NextK_3_EMAP = con_bin_str(NextK_3_int_EMAP)
    NextK_4_EMAP = con_bin_str(NextK_4_int_EMAP)
    
    NextIDP_K_EMAP = NextIDP_EMAP + NextK_1_EMAP + NextK_2_EMAP + NextK_3_EMAP + NextK_4_EMAP
    
                               
    return A_EMAP, B_EMAP, C_EMAP, D_EMAP, E_EMAP, NextIDP_K_EMAP


IndentationError: expected an indented block (Temp/ipykernel_24664/838223077.py, line 7)

In [98]:
class EMAPoracle2:
    def __init__(self, size):
        temp = rand_bitstream(size*5)
        self.idp_emap_str, self.k1_str, self.k2_str, self.k3_str, self.k4_str = [temp[i:i+size] for i in range(0,len(temp),size)]
        
        self.idp_int = int(self.idp_emap_str,2)
        
        self.ID = int('101110011011001011100100001101010011110010001001000101001101001001110010110100100001011001110010',2)
                
        #create randomized n values 
        self.n1 = int(rand_bitstream(size),2)
        self.n2 = int(rand_bitstream(size),2)
        
        #create the integer Ks in here
        self.k1 = int(self.k1_str,2)
        self.k2 = int(self.k2_str,2)
        self.k3 = int(self.k3_str,2)
        self.k4 = int(self.k4_str,2)
        
        self.mod = 2**size
        
    def helloInternal():
        
        id_str1, id_str2 = [self.idp_emap_str[i:i+48] for i in range(0,len(self.id_emap_str),48)]
        
        k1fp = Fp_func(self.k1_str)
        k2fp = Fp_func(self.k2_str)
        k3fp = Fp_func(self.k3_str)
        k4fp = Fp_func(self.k4_str)
        
        sub_new_k1 = (id_str1 + k4fp + k3fp)
        sub_new_k2 = (k1fp + k4fp + id_str2)
        sub_new_k3 = (id_str1 + k4fp + k2fp)
        sub_new_k4 = (k3fp + k1fp + id_str2)
        
        new_k1int = int(sub_new_k1,2)
        new_k2int = int(sub_new_k2,2)
        new_k3int = int(sub_new_k3,2)
        new_k4int = int(sub_new_k4,2)
        
        new_idp = con_bin_str((self.idp_int ^ self.n2 ^ self.k1)%self.mod)
        newk1 = con_bin_str((self.k1 ^ self.n2 ^ new_k1int)%self.mod)
        newk2 = con_bin_str((self.k2 ^ self.n2 ^ new_k2int)%self.mod)
        newk3 = con_bin_str((self.k3 ^ self.n1 ^ new_k3int)%self.mod)
        newk4 = con_bin_str((self.k4 ^ self.n1 ^ new_k4int)%self.mod)
        
        newIDP = new_idp + newk1 + newk2 + newk3 + newk4
        return newIDP
        

In [99]:
oracle = EMAPoracle2(96)

def hello(oracle):
    return oracle.helloInternal()   
  
#take in oracle, return a, b, c and oracle (idp)     
def layer2(oracle):
        
    a_int = (oracle.idp_int ^ oracle.k1 * oracle.n1)%oracle.mod
    b_int = ((oracle.idp_int | oracle.k2)^oracle.n1)%oracle.mod
    c_int = (oracle.idp_int ^ oracle.k3 ^ oracle.n2)%oracle.mod
    
    a = con_bin_str(a_int)
    b = con_bin_str(b_int)
    c = con_bin_str(c_int)
    
    return a, b, c, oracle
        
def layer3(a, b, c, oracle):
    #calc n1, n2
    n1 = a ^ oracle.idp_int ^ oracle.k1
    n2 = c ^ oracle.idp_int ^ oracle.k3    
    
    
    ki = oracle.k1 ^ oracle.k2 ^ oracle.k3 ^ oracle.k4
        
    d_int = ((oracle.idp_int & oracle.k4) ^ n2)%oracle.mod 
    e_int = ((oracle.idp_int & n1 | n2) ^ oracle.ID ^ ki)%oracle.mod
    
    d = con_bin_str(d_int)
    e = con_bin_str(e_int)
        
    return d, e

print(layer2(oracle))

idp:  000111000101011101100111101110111111011101001111000111110010011101101111000100110100000110101101
k1:  6683476058386139716142683652
('111000101011000011101011011101000011100001010001011011100101111100101010111100101110111010110001', '000010001011110001010110100101111001001011000100100000101010100000111010111100010110110011101010', '100100000101010000110011100001100010001110110111100011000101010011110000101001111100100011110001', <__main__.EMAPoracle2 object at 0x00000227EDA59A30>)


In [ ]:
#Finds and outputs the ID of an MMAP RFID system, given only intercepted comms between reader and tag
SIZE = 96
def EMAP_Cracker():
    garbage = "" #Used to collect useless information, and ignore it
    
    #Finding the first IDP we will use, so that it works from any random valid IDP
    #We don't care about the previous A-E, since we are assuming that we don't know the IDP used for that interaction
    garbage,garbage,garbage,garbage,garbage,IDP_next = EMAP(IDP_K_str)
    
    #Initializing our ID output to all 9s, to represent that we don't know what it is
    ID = np.array([9]*SIZE)
    
    #Until we return a complete ID, keep making MMAP exchanges
    while(1):
        #Initializing our IDP
        IDP = IDP_next[0:SIZE]
        #Initializing A-E, and the next IDP
        A,B,C,D,E,IDP_next = MMAP(IDP_next)
        
        #Initializing n1 and n2 to all 9s
        n1 = np.array([9]*SIZE)
        n2 = np.array([9]*SIZE)
        
        #Finding all knowable values of n1
        i = 0
        for i in range(SIZE):
            if IDP[i] == '0':
                if B[i] == '0':
                    n1[i] = 0
                else:
                    n1[i] = 1
       # npPrint(n1)
        #Creating our negative IDP in two's complement
        negIDP = ""
        for i in range(SIZE):
            if IDP[i] == "0":
                negIDP += "1"
            else:
                negIDP += "0"
        negIDP = bin(int(negIDP,2)+1)[2:].zfill(SIZE)
#         print("IDP:    ",IDP)
#         print("negIDP: ",negIDP)
#         print("IDPCHK: ", bin(int(IDP,2)+int(negIDP,2)))
        
        #Initializing E XOR n1 to 9s
        E_xor_n1 = np.array([9]*SIZE)
        
        #Calculating knowable values of E XOR n1
    #    print(E)
        for i in range(SIZE):
            num = int(E[i],2)
            if(n1[i] != 9):
                E_xor_n1[i] = num^n1[i]
    #    npPrint(E_xor_n1)
        
        carry = 0
        for i in range(SIZE-1,-1,-1):
            if ID[i] == 9: break
            
            # negIDP 111
            # EXOR 001
            # ID 000
            
            # ID 000
            # negIDP 111
            # EXOR 111 WRONG
            val = ID[i] + int(negIDP[i],2) + carry
            if E_xor_n1[i] == 9:
                E_xor_n1[i] = (val)%2
            carry = E_xor_n1[i] + int(negIDP[i],2) + carry > 1
        #print("E_xor_n1 predic: ",npPrint(E_xor_n1))
        
        #Calculating knowable values of the ID
        carry = 0
        for i in range(SIZE-1,-1,-1):
            if E_xor_n1[i] == 9: break
            val = E_xor_n1[i] + int(negIDP[i],2) + carry
            if ID[i] == 9:
                ID[i] = (val)%2
            carry = val > 1
        #If ID's most significant bit is found, return it
        if ID[0] != 9:
            return ID
MMAP_Cracker()

In [14]:
a, b, c, d, e, oracle_new = EMAP(IDP_K_str)


